In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import plotly.graph_objects as go

In [2]:
import pandas_datareader as web
import datetime as dt

In [31]:
class StockModel:
    
    def __init__(self,stk_data,prediction_days = 30):
        self.data, self.test_data = train_test_split(stk_data, test_size=.2, shuffle=False)
        self.scaler = MinMaxScaler(feature_range=(0,1))
        self.scaled_data = self.scaler.fit_transform(self.data['Close'].values.reshape(-1,1))
        
        self.x_train = []
        self.y_train = []
        for x in range(prediction_days, len(self.scaled_data)):
            self.x_train.append(self.scaled_data[x-prediction_days:x, 0])
            self.y_train.append(self.scaled_data[x, 0])
        self.x_train, self.y_train = np.array(self.x_train), np.array(self.y_train)
        print(self.x_train.shape)
        print(self.x_train.shape)
        
        self.x_train = np.reshape(self.x_train, (self.x_train.shape[0], self.x_train.shape[1], 1))
        #test data preparation
        self.actual_prices = self.test_data['Close'].values
        self.total_dataset = pd.concat((self.data['Close'], self.test_data['Close']), axis=0)
        self.model_inputs = self.total_dataset[len(self.total_dataset) - len(self.test_data) - prediction_days:].values
        self.model_inputs = self.model_inputs.reshape(-1,1)
        self.model_inputs = self.scaler.transform(self.model_inputs)
        self.x_test = []
        for x in range(prediction_days, len(self.model_inputs)):
            self.x_test.append(self.model_inputs[x-prediction_days:x, 0])
        self.x_test = np.array(self.x_test)
        self.x_test = np.reshape(self.x_test, (self.x_test.shape[0], self.x_test.shape[1], 1))
        return

    def create(self):
        self.model = Sequential()
        self.model.add(LSTM(units=50, return_sequences=True, input_shape=(self.x_train.shape[1], 1)))
        self.model.add(Dropout(0.2))
        self.model.add(LSTM(units=50, return_sequences=True))
        self.model.add(Dropout(0.2))
        self.model.add(LSTM(units=50))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(units=1))
        self.model.compile(optimizer='adam', loss='mean_squared_error')
        self.model.fit(self.x_train, self.y_train, epochs=25, batch_size=32)
        return self.model
        
    def predict(self, model):
        self.model = model
        self.predicted_prices = self.model.predict(self.x_test)
        self.predicted_prices = self.scaler.inverse_transform(self.predicted_prices)

    def plotting(self):
        data = pd.DataFrame()
        data['Date'] = self.test_data['Date']
        data['actual'] = self.actual_prices
        data['prediction'] = self.predicted_prices
        trace1 = go.Scatter(x=data.Date, y=data.actual, text='actual prices')
        trace2 = go.Scatter(x=data.Date, y=data.prediction, text='predicted prices')
        figure = [trace1, trace2]
        fig = {'data':figure}
        return fig

    def results(self):
        prediction_days = 60
        self.real_data = [self.model_inputs[len(self.model_inputs)+1-prediction_days:len(self.model_inputs+1), 0]]
        self.real_data = np.array(self.real_data)
        self.real_data = np.reshape(
        self.real_data, (self.real_data.shape[0], self.real_data.shape[1], 1))
        prediction = self.model.predict(self.real_data)
        prediction = self.scaler.inverse_transform(prediction)
        r2 = round(r2_score(self.actual_prices, self.predicted_prices)*100)
        return {'r2': r2,'prediction': prediction[0][0]}


In [34]:
company = 'FB'

In [35]:
start = dt.datetime(2000,1,1)
end = dt.datetime.now()

In [36]:
data = web.DataReader(company, 'yahoo', start, end)
data.to_csv('stk_data.csv',index=True)

In [37]:
stk_data = pd.read_csv('stk_data.csv')
stk_data

,Date,High,Low,Open,Close,Volume,Adj Close
0,2012-05-18,45.000000,38.000000,42.049999,38.230000,573576400,38.230000
1,2012-05-21,36.660000,33.000000,36.529999,34.029999,168192700,34.029999
2,2012-05-22,33.590000,30.940001,32.610001,31.000000,101786600,31.000000
3,2012-05-23,32.500000,31.360001,31.370001,32.000000,73600000,32.000000
4,2012-05-24,33.209999,31.770000,32.950001,33.029999,50237200,33.029999
...,...,...,...,...,...,...,...
2261,2021-05-14,316.850006,309.079987,309.540009,315.940002,19230600,315.940002
2262,2021-05-17,315.679993,311.579987,313.549988,315.459991,15401300,315.459991
2263,2021-05-18,316.500000,309.809998,315.589996,309.959991,14522400,309.959991
2264,2021-05-19,314.660004,303.570007,304.190002,313.589996,19106200,313.589996


In [38]:
nn = StockModel(stk_data,60)

(1752, 60)
(1752, 60)


In [39]:
seq = nn.create()
seq.save(f'models/{company}_nn')

Epoch 1/25
55/55 [==============================] - 11s 194ms/step - loss: 0.0215
Epoch 2/25
55/55 [==============================] - 8s 138ms/step - loss: 0.0034
Epoch 3/25
55/55 [==============================] - 7s 134ms/step - loss: 0.0033
Epoch 4/25
55/55 [==============================] - 8s 140ms/step - loss: 0.0030
Epoch 5/25
55/55 [==============================] - 8s 137ms/step - loss: 0.0031
Epoch 6/25
55/55 [==============================] - 7s 136ms/step - loss: 0.0029
Epoch 7/25
55/55 [==============================] - 7s 133ms/step - loss: 0.0028
Epoch 8/25
55/55 [==============================] - 7s 136ms/step - loss: 0.0028
Epoch 9/25
55/55 [==============================] - 7s 134ms/step - loss: 0.0028
Epoch 10/25
55/55 [==============================] - 7s 133ms/step - loss: 0.0024
Epoch 11/25
55/55 [==============================] - 7s 129ms/step - loss: 0.0023
Epoch 12/25
55/55 [==============================] - 7s 130ms/step - loss: 0.0024
Epoch 13/25
55/55 [=====

In [40]:
from tensorflow import keras
model = keras.models.load_model('models\RELI_nn')

In [41]:
nn.predict(model)

In [42]:
nn.results()

{'r2': -464, 'prediction': 167.37987}

In [43]:
f= go.Figure()
figs = nn.plotting()
f.add_trace(figs['data'][0])
f.add_trace(figs['data'][1])
f.show()

In [45]:
def save(tik,year=2012):
    start = dt.datetime(year,1,1)
    end = dt.datetime.now()
    data = web.DataReader(tik, 'yahoo', start, end)
    data.to_csv('stk_data.csv', index=True)
    nn = StockModel(data,60)
    seq = nn.create()
    seq.save(f'models/{tik}_nn')

In [46]:
stks = {
    'Facebook':'FB',
    'Apple':'AAPL',
    'Tesla':'TSLA',
    'Amazon':'AMZN',
    'Google':'GOOG',
    'Microsoft Corporation':'MSFT',
    'Tata Motors Limited':'TTM',
    'Netflix, Inc.':'NFLX',
    'NVIDIA Corporation':'NVDA',
    'Wipro Limited':'WIPRO.NS',
    'Dogecoin USD':'DOGE-USD',
    'EthereumClassic USD':'ETC-USD',
    'BitcoinCash USD':'BCH-USD',
    'QuantumScape Corporation':'QS',
    'Virgin Galactic Holdings, Inc.':'SPCE',
    'Dogecoin INR':'DOGE-INR',
    'The Toro Company':'TTC',
    'Churchill Capital Corp IV':'CCIV',
    'Plug Power Inc.':'PLUG',
    'Singapore Telecommunications Limited':'Z74.SI',
    'FuelCell Energy, Inc.':'FCEL',
    }
for company in stks.values():
    save(company)

0023
Epoch 16/25
43/43 [==============================] - 7s 155ms/step - loss: 0.0023
Epoch 17/25
43/43 [==============================] - 7s 164ms/step - loss: 0.0026
Epoch 18/25
43/43 [==============================] - 7s 161ms/step - loss: 0.0022
Epoch 19/25
43/43 [==============================] - 7s 165ms/step - loss: 0.0019
Epoch 20/25
43/43 [==============================] - 7s 157ms/step - loss: 0.0022
Epoch 21/25
43/43 [==============================] - 7s 153ms/step - loss: 0.0020
Epoch 22/25
43/43 [==============================] - 6s 150ms/step - loss: 0.0019
Epoch 23/25
43/43 [==============================] - 7s 154ms/step - loss: 0.0024
Epoch 24/25
43/43 [==============================] - 7s 152ms/step - loss: 0.0021
Epoch 25/25
43/43 [==============================] - 7s 156ms/step - loss: 0.0017
INFO:tensorflow:Assets written to: models/ETC-USD_nn\assets
(1056, 60)
(1056, 60)
Epoch 1/25
33/33 [==============================] - 4s 126ms/step - loss: 0.0064
Epoch 2/25
3

In [47]:
def save_results(data,sentiment):
    sess = opendb()
    obj = prediction(comment=data,result=sentiment)
    sess.commit()
    sess.close()